# support vector machine

## 必要そうなライブラリのインポート

In [21]:
import os
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import sklearn.metrics as metrics
import librosa
import optuna

from functools import partial

## ラベル名取得、.wavファイルのリスト、24次元MFCC取得の関数

In [22]:
def get_labelname(Tr):
    """[summary]
    training下のラベル名を取得
    Returns:
        [type]: [description]
    """    
    current = os.getcwd()
    if Tr == False:
        filepath = current + '/train/'
    elif Tr == True:
        filepath = current + '/test/'
    print(filepath)
    labellist = []
    for dir in os.listdir(filepath):
        if os.path.isdir(os.path.join(filepath, dir))==True:
            labellist.append(dir)
    
    return labellist, filepath

def wav2list(p):
    """[summary]
    Get audio file list to process all at once
    Returns:
        list : list of audio path
    """
    p = Path(p)
    audio_list = list(p.glob('*.wav'))

    if len(audio_list) == 0:
        sys.exit('Not found in {}'.format(p))

    return audio_list

def get_mfcc_librosa(p):
    """[summary]
    librosaライブラリを用いて24次元MFCCを抽出する
    データはtraining以下に置き, 各ラベルごとにフォルダを作ってデータを置いておく
    Args:
        p ([str]): .wavデータが置いてあるディレクトリ名
    Returns:
        [tupple]: (ファイル名, 24次元のMFCC)
    """    
    wavlist = wav2list(p)
    _name = []
    _mfcc = []
    wavlist.sort()

    for wavfile in wavlist:
        y, sr = librosa.core.load(wavfile,sr=44100)
        tmp = librosa.feature.mfcc(y=y, sr=44100, hop_length=10, win_length=100, n_mfcc=24)
        ceps = tmp.mean(axis=1)
        # print(ceps)
        _name.append(wavfile.stem)
        _mfcc.append(ceps)

    return _name, _mfcc

## Training dataのデータフレームの作成

In [23]:
def make_df(Tr):
    """[summary]
    trainディレクトリ下から.wavデータを取ってきて抽出したMFCCとラベルから構成されるデータフレームを作成
    Returns:
        df_new[dataframe]: 学習データのデータフレーム
    """    
    labellist, filepath = get_labelname(Tr)
    print(labellist)
    cols = [x for x in range(24)]
    print(cols)
    cols.append('label')
    print(cols)
    df_new = pd.DataFrame(index = [], columns=cols)
    # print(df_new)
    for label in labellist:
        # print(filepath, label)
        labelpath = filepath + label
        filename, tmp = get_mfcc_librosa(labelpath)
        df = pd.DataFrame(tmp, index=filename)
        df = df.assign(label=label)
        df_new = pd.concat([df_new, df], axis=0)
    
    return df_new

In [24]:
# Training dataのデータフレーム作成 
df = make_df(Tr=True)
# x : 24次元のMFCC特徴量, y : データのラベル
x = df.iloc[:, 0:24]
y = df.iloc[:, 24]

## ラベルを数値に変換

In [25]:
label = set(y)
label_list = list(label)
# label_list.sort()

for i in range(len(label_list)):
    y[y == label_list[i]] =i

y = np.array(y, dtype = "int")
print(len(y))
y

## Test dataのデータフレーム作成

In [26]:
df_test = make_df(Tr=False)
# x : 24次元のMFCC特徴量, y : データのラベル
x_t = df_test.iloc[:, 0:24]
y_t = df_test.iloc[:, 24]

## ラベルを数値に変換

In [27]:
label_test = set(y_t)
label_list_test = list(label_test)

for i in range(len(label_list_test)):
    y_t[y_t == label_list_test[i]] =i

y_t = np.array(y_t, dtype = "int")
print(len(y_t))
y_t

## データを標準化して学習

In [28]:
sc = StandardScaler()
sc.fit(x)
x_train_std = sc.transform(x)
x_test_std = sc.transform(x_t)

## Optunaを用いてハイパーパラメータを調整

In [ ]:
def opt(X, y, trial):
    C = trial.suggest_loguniform('C', 1, 100)
    gamma = trial.suggest_loguniform('gamma', 0.001, 3.0)
    model_rbf = OneVsRestClassifier(SVC(
                C = C,
                kernel = 'rbf',
                gamma = gamma,
                random_state = 42,
                class_weight = "balanced",
    ))
    # 5分割してCrosss Validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    '''
    model_rbf = OneVsRestClassifier(model_rbf)
    model_rbf.fit(x_train_std, y)
    tuna_pred_test = model_rbf.predict(x_test_std)
    '''
    
    scores = cross_validate(model_rbf, X=X, y=y, cv=kf, scoring=['f1_weighted'])
    scores = cross_validate(model_rbf, X=X, y=y, cv=kf)
    return (1.0 - scores['test_f1_weighted'].mean())

In [29]:
def opt(trial):
    C = trial.suggest_loguniform('C', 1, 100)
    gamma = trial.suggest_loguniform('gamma', 0.001, 3.0)
    model_rbf = SVC(
                C = C,
                kernel = 'rbf',
                gamma = gamma,
                random_state = 42,
                class_weight = "balanced",
    )
    
    model_rbf = OneVsRestClassifier(model_rbf)
    model_rbf.fit(x_train_std, y)
    tuna_pred_test = model_rbf.predict(x_test_std)
    return (1.0 - f1_score(tuna_pred_test, y_t, average=None).mean())

In [30]:
#セッション作成
# f = partial(opt, x_train_std, y)
study = optuna.create_study()
#回数
study.optimize(opt, n_trials=100)

In [31]:
print(study.best_params)
print(study.best_value)
print(study.best_trial)

## モデルの学習時間計測

In [24]:
model_time = SVC()

In [18]:
%%time
model_time.fit(x_train_std, y)

## 学習データの結果(Accuracy, Precision, Recall, F1)

In [32]:
model_rbf = SVC(C = study.best_params["C"], gamma = study.best_params["gamma"], kernel = "rbf", random_state = 42, class_weight='balanced')
model_rbf = OneVsRestClassifier(model_rbf)

In [33]:
model_rbf = model_rbf.fit(x_train_std, y)

In [34]:
pred_rbf_train = model_rbf.predict(x_train_std)

accuracy_rbf_train = accuracy_score(y, pred_rbf_train)
precision_rbf_train = precision_score(y, pred_rbf_train, average=None)
recall_rbf_train = recall_score(y, pred_rbf_train, average=None)
f1_rbf_train = f1_score(y, pred_rbf_train, average='weighted')

In [35]:
print("train_result")
print("Accuracy : "+ str(accuracy_rbf_train))
print("Precision : "+ str(precision_rbf_train))
print("Recall : "+ str(recall_rbf_train))
print("F1 : "+ str(f1_rbf_train))

## テストデータの結果(Accuracy, Precision, Recall, F1)

In [36]:
%%time
pred_rbf_test = model_rbf.predict(x_test_std)

In [37]:
accuracy_rbf_test = accuracy_score(y_t, pred_rbf_test)
precision_rbf_test = precision_score(y_t, pred_rbf_test, average=None)
recall_rbf_test = recall_score(y_t, pred_rbf_test, average=None)
f1_rbf_test = f1_score(y_t, pred_rbf_test, average='weighted')
f1_rbf = f1_score(y_t, pred_rbf_test, average=None)

In [38]:
pred_rbf_test

## 学習ecopa, テストarena

In [19]:
print("test_result")
print("Accuracy : "+ str(accuracy_rbf_test))
print("Precision : "+ str(precision_rbf_test))
print("Recall : "+ str(recall_rbf_test))
print("F1 : "+ str(f1_rbf))
print("F1_rbf : "+ str(f1_rbf.mean()))

In [20]:
confusion = confusion_matrix(y_t, pred_rbf_test)
confusion

In [82]:
print("test_result")
print("Accuracy : "+ str(accuracy_rbf_test))
print("Precision : "+ str(precision_rbf_test))
print("Recall : "+ str(recall_rbf_test))
print("F1 : "+ str(f1_rbf))
print("F1 mean: "+ str(f1_rbf.mean()))

In [83]:
confusion = confusion_matrix(y_t, pred_rbf_test)
confusion

## 学習arena, テストecopa

In [39]:
print("test_result")
print("Accuracy : "+ str(accuracy_rbf_test))
print("Precision : "+ str(precision_rbf_test))
print("Recall : "+ str(recall_rbf_test))
print("F1 : "+ str(f1_rbf))
print("F1_mean : "+ str(f1_rbf.mean()))

In [40]:
confusion = confusion_matrix(y_t, pred_rbf_test)
confusion